## Updating data about countries

### Importing packages for parsing and working with db

In [1]:
from bs4 import BeautifulSoup as BS
from aiohttp import ClientSession
import psycopg2 as pg
import psycopg2.extras
import requests
import re
import csv
import pandas as pd
import time
import random

### Connecting to db

In [40]:
global base, cur
base = pg.connect(
    host='*',
    dbname='*',
    user='*',
    password='*',
    port=*)
cur = base.cursor()

### Close connection if it's needed

In [28]:
base.close()
cur.close()

### Getting list of countries

In [88]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) '
                         'Chrome/39.0.2171.95 Safari/537.36'}
async with ClientSession() as session:
    async with session.get("https://simple.wikipedia.org/wiki/List_of_countries",
                           headers=headers) as countries_wiki:
        html = BS(await countries_wiki.text(), 'lxml')
        data = html.find('div', class_='mw-parser-output')
        countries = []
        close_loop = False
        for a in data.find_all('p'):
            for i in a.find_all('a', href=True):
                if i.get("title").lower() == "abkhazia":
                    close_loop = True
                    break
                if close_loop == True:
                    break
                countries.append(i.get("title").lower())
countries.sort()

In [35]:
print(countries)

['afghanistan', 'albania', 'algeria', 'andorra', 'angola', 'antigua and barbuda', 'argentina', 'armenia', 'australia', 'austria', 'azerbaijan', 'bahrain', 'bangladesh', 'barbados', 'belarus', 'belgium', 'belize', 'benin', 'bhutan', 'bolivia', 'bosnia and herzegovina', 'botswana', 'brazil', 'brunei', 'bulgaria', 'burkina faso', 'burundi', 'cambodia', 'cameroon', 'canada', 'cape verde', 'central african republic', 'chad', 'chile', 'china', 'colombia', 'comoros', 'costa rica', 'croatia', 'cuba', 'cyprus', 'czech republic', 'democratic republic of the congo', 'denmark', 'djibouti', 'dominica', 'dominican republic', 'east timor', 'ecuador', 'egypt', 'el salvador', 'equatorial guinea', 'eritrea', 'estonia', 'eswatini', 'ethiopia', 'federated states of micronesia', 'fiji', 'finland', 'france', 'gabon', 'georgia (country)', 'germany', 'ghana', 'greece', 'grenada', 'guatemala', 'guinea', 'guinea-bissau', 'guyana', 'haiti', 'honduras', 'hungary', 'iceland', 'india', 'indonesia', 'iran', 'iraq', 

In [36]:
data = []
for country in countries:
    country_second_name = ""
    country_third_name = ""
    if country == "united states":
        country_second_name = "us"
        country_third_name = "usa"
    elif country == "united kingdom":
        country_second_name = "uk"
    elif country == "united arab emirates":
        country_second_name = "uae"
    elif country == "turkey":
        country_second_name = "turkiye"
        country_third_name = "türkiye"
    elif country == "republic of china":
        country_second_name = "china"
    elif country == "russia":
        country_second_name = "russian federation"
    elif country == "egypt":
        country_second_name = "egypt arab rep"
    elif country == "georgia (country)":
        country_second_name = "georgia"
    elif country == "iran":
        country_second_name = "iran islamic rep"
    elif country == "ivory coast":
        country_second_name = "cote divoire"
    elif country == "kingdom of the netherlands":
        country_second_name = "netherlands"
    elif country == "kyrgyzstan":
        country_second_name = "kyrgyz republic"
    elif country == "laos":
        country_second_name = "lao pdr"
    elif country == "vietnam":
        country_second_name = "viet nam"
    elif country == "south korea":
        country_second_name = "korea rep"
    elif country == "north korea":
        country_second_name = "korea dem peoples rep"
    elif country == "slovakia":
        country_second_name = "slovak republic"
    elif country == "democratic republic of the congo":
        country_second_name = "congo dem rep"
    elif country == "republic of the congo":
        country_second_name = "congo rep"
    elif country == "syria":
        country_second_name = "syrian arab republic"
    elif country == "yemen":
        country_second_name = "yemen rep"
    elif country == "brunei":
        country_second_name = "brunei darussalam"
    elif country == "east timor":
        country_second_name = "timor leste"
    elif country == "federated states of micronesia":
        country_second_name = "micronesia fed sts"
    elif country == "republic of ireland":
        country_second_name = "ireland"
    elif country == "saint kitts and nevis":
        country_second_name = "st kitts and nevis"
    elif country == "saint vincent and the grenadines":
        country_second_name = "st vincent and the grenadines"
    elif country == "the bahamas":
        country_second_name = "bahamas the"
    elif country == "the gambia":
        country_second_name = "gambia the"
    data.append((country, country_second_name, country_third_name))

### Now finding and writing data about countries

In [37]:
for idx, item in enumerate(data):
       
    country = item[0]
    headers = {
            'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) '
                          'Chrome/39.0.2171.95 Safari/537.36'}
    async with ClientSession() as session:
        async with session.get("https://en.wikipedia.org/wiki/" + country.replace(" ", "_"),
                               headers=headers) as countries_wiki:
            html = BS(await countries_wiki.text(), 'lxml')
            info = html.find('table', class_=re.compile(r'^infobox'))
            if info is None:
                print("didn't get info about " + country)
                continue
            img = None
            capital = ""
            largest_city = ""
            official_languages = ""
            common_languages = ""
            currency = ""
            time_zone = ""
            calling_code = ""
            
            for i in info.find_all('tr'):
                
                tag_to_exclude = info.find_all('sup')
                for tag in tag_to_exclude:
                    tag.decompose()
                    
                if i.find('th', class_='infobox-label') is not None:
                    current_str = i.find('th', class_='infobox-label').text.replace(u'\xa0', ' ')
                    if current_str.find("Capital") != -1:
                        if i.find('a') is None:
                            capital = i.text
                        else:
                            capital = i.find('a').text
                    elif current_str == "Largest city":
                        if i.find('a') is None:
                            largest_city = i.text
                        else:
                            largest_city = i.find('a').text
                    elif current_str.find("Official") != -1 and current_str.find("lang") != -1:
                        if i.find('td').find('li') is not None:
                            for links in i.find('td').find_all('li'):
                                official_languages = links.text if  official_languages == "" else official_languages + ", " + links.text
                        else:
                            official_languages = i.find('td').text
                    elif current_str.find("National") != -1 and current_str.find("lang") !=-1 and official_languages == "":
                        if i.find('td').find('li') is not None:
                            for links in i.find('td').find_all('li'):
                                official_languages = links.text if  official_languages == "" else official_languages + ", " + links.text
                        else:
                            official_languages = i.find('td').text
                    elif current_str == "Common languages" or current_str == "Spoken languages":
                        if i.find('td').find('li') is not None:
                            for links in i.find('td').find_all('li'):
                                common_languages = links.text if common_languages == "" else  common_languages + ", " + links.text
                        else:
                            common_languages = i.find('td').text
                    elif current_str == "Currency":
                        if i.find('td').find('li') is not None:
                            for links in i.find('td').find_all('li'):
                                currency = links.text if currency == "" else currency + ", " + links.text
                        else:
                            currency = i.find('td').text
                    elif current_str == "Time zone":
                        time_zone =  i.find('td').text
                    elif current_str == "Calling code":
                        calling_code =  i.find('td').text
   
            img = "https:" + info.find_all('img')[0].get('src')
        data[idx] = item + (img, capital, largest_city, official_languages, common_languages, currency, time_zone, calling_code)

didn't get info about saint lucia
didn't get info about sovereign state


In [38]:
print(data)

[('afghanistan', '', '', <psycopg2.extensions.Binary object at 0x000001F748E06F10>, 'Kabul', '', 'Pashto, Dari', '', 'Afghani (افغانى) (AFN)', 'UTC+4:30Lunar Calendar (Afghanistan Time)', ''), ('albania', '', '', <psycopg2.extensions.Binary object at 0x000001F74B8C33C0>, 'Tirana', '', 'Albanian', '', 'Lek (ALL)', 'UTC+1 (CET)', '+355'), ('algeria', '', '', <psycopg2.extensions.Binary object at 0x000001F74C690E70>, 'Algiers', '', 'Arabic, Berber', '', 'Algerian dinar (DZD)', 'UTC+1 (CET)', '+213'), ('andorra', '', '', <psycopg2.extensions.Binary object at 0x000001F74A04FB10>, 'Andorra la Vella', '', 'Catalan ', '', 'Euro (€) (EUR)', 'UTC+01 (CET)', '+376'), ('angola', '', '', <psycopg2.extensions.Binary object at 0x000001F74E292700>, 'Luanda', '', 'Portuguese', '', 'Angolan kwanza (AOA)', 'UTC+1 (WAT)', '+244'), ('antigua and barbuda', '', '', <psycopg2.extensions.Binary object at 0x000001F74CE12100>, "St. John's", '', '', '', 'East Caribbean dollar (XCD)', 'UTC-4 (AST)', '+1-268'), ('a

In [39]:
for idx, item in enumerate(data):
    
    user_agents = [
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
    'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.81 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.81 Safari/537.36',
    ]
    
    if idx % 20 == 0:
        time.sleep(3*60) # waiting for 3 mins every 20 strings because site can block connections

    headers = {
            'User-Agent': random.choice(user_agents)}
    async with ClientSession() as session:
        async with session.get("https://data.worldbank.org/country/" + item[0].replace(" ", "-"),
                               headers=headers) as country_twb:
            if country_twb.status != 200:
                async with session.get("https://data.worldbank.org/country/" + item[1].replace(" ", "-"),
                               headers=headers) as country_twb:
                    if country_twb.status != 200:
                        async with session.get("https://data.worldbank.org/country/" + item[2].replace(" ", "-"),
                                   headers=headers) as country_twb:
                            html = BS(await country_twb.text(), 'lxml')
                    else:
                        html = BS(await country_twb.text(), 'lxml')
            else:
                html = BS(await country_twb.text(), 'lxml')
                
            population = ""
            net_migration = ""
            gdp_per_capita = ""
            gdp_growth = ""
            unemployment = ""
            inflation = ""
            co2_emissions = ""
            access_to_electricity = ""
            electricity_production = ""
            access_to_internet = ""
            
            for info in html.find_all('div', class_='indicator-item'):
                for i in info.find_all('div', class_='indicator-item__inner'):
                    if i.find('div', class_='indicator-item__title') is not None:
                        try:
                            current_str = i.find('div', class_='indicator-item__title').text.replace(u'\xa0', ' ')
                            if current_str.find("Population, total") != -1:
                                population = i.find('div', class_='indicator-item__data-info').text
                            elif current_str.find("Net migration") != -1:
                                net_migration = i.find('div', class_='indicator-item__data-info').text
                            elif current_str.find("GDP per capita") != -1:
                                gdp_per_capita = i.find('div', class_='indicator-item__data-info').text
                            elif current_str.find("GDP growth") != -1:
                                gdp_growth = i.find('div', class_='indicator-item__data-info').text + "%"
                            elif current_str.find("Unemployment, total") != -1:
                                unemployment = i.find('div', class_='indicator-item__data-info').text + "%"
                            elif current_str.find("Inflation") != -1:
                                inflation = i.find('div', class_='indicator-item__data-info').text + "%"
                            elif current_str.find("CO2 emissions") != -1:
                                co2_emissions = i.find('div', class_='indicator-item__data-info').text
                            elif current_str.find("Access to electricity") != -1:
                                access_to_electricity = i.find('div', class_='indicator-item__data-info').text + "%"
                            elif current_str.find("Electricity production") != -1:
                                electricity_production = i.find('div', class_='indicator-item__data-info').text + "%"
                            elif current_str.find("Individuals using the Internet") != -1:
                                access_to_internet = i.find('div', class_='indicator-item__data-info').text + "%"
                        except:
                            continue
                            
        data[idx] = item + (population, net_migration, gdp_per_capita, gdp_growth, unemployment, inflation, co2_emissions, access_to_electricity, electricity_production, access_to_internet)

In [40]:
#Global piece index
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36',
    'Referer': 'https://www.visionofhumanity.org/',
}
async with ClientSession() as session:
    async with session.get("https://www.visionofhumanity.org/maps/",
                           headers=headers) as countries_wiki:
        html = BS(await countries_wiki.text(), 'lxml')
        div_maps = html.find('section', re.compile(r'^map'))
        links = div_maps.find('div', attrs={'data-datasets': True})
        if links is not None:
            # Get all strings finished with .csv 
            csv_links = [link.strip("'") for link in str(links['data-datasets']).split(',')]
            if csv_links != []:
                full_csv_url = f"https://www.visionofhumanity.org{csv_links[-1]}"

                # Now fetch the CSV file
                async with session.get(full_csv_url) as csv_response:
                    csv_content = await csv_response.text()

                    # Read the CSV content
                    csv_lines = csv_content.splitlines()
                    for idx, item in enumerate(data):
                        rank = ''
                        for row in csv.DictReader(csv_lines):
                            if row['name'].lower() in item or row['code'].lower() in item:
                                rank = 'N/A' if row['rank'] == "-1" else row['rank']
                                break
                        data[idx] = item + (rank,)

In [41]:
#Freedom score
freedom_score = []

headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) '
                      'Chrome/39.0.2171.95 Safari/537.36'}
async with ClientSession() as session:
    async with session.get("https://freedomhouse.org/countries/freedom-world/scores",
                           headers=headers) as countries_wiki:
        html = BS(await countries_wiki.text(), 'lxml')
        table = html.find('table', class_='responsive-enabled').find('tbody')
        for row in table.find_all('tr'):
            if row is not None:
                country = row.find_all('td')[0].text.replace("*", "")
                total_score_and_status = row.find_all('td')[1].text
                political_rights = row.find_all('td')[2].text
                civil_liberties = row.find_all('td')[3].text
                freedom_score.append((country, total_score_and_status, political_rights, civil_liberties))
        #writing data keeping same quantity of elements for every tuple
        for idx, item in enumerate(data):
            total_score_and_status = ""
            political_rights = ""
            civil_liberties = ""
            for lst in freedom_score:
                if lst[0].lower() in item:
                    total_score_and_status = lst[1]
                    political_rights = lst[2]
                    civil_liberties = lst[3]
                    break
            data[idx] = item + (total_score_and_status, political_rights, civil_liberties)

In [42]:
#Press freedom index
pfi = []
headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) '
                      'Chrome/39.0.2171.95 Safari/537.36'}
async with ClientSession() as session:
    async with session.get("https://rsf.org/en/index/",
                           headers=headers) as countries_wiki:
        html = BS(await countries_wiki.text(), 'lxml')
        info = html.find('div', class_='country-list')
        for items in info.find_all('div', class_='country-list-item'):
            country = items.find('span', class_='country-list-item__name').text
            rank = items.find('span', class_='country-list-item__score').text + " (world place " + items.find('span', class_='country-list-item__position').text + ")"
            pfi.append((country, rank))
        #writing data keeping same quantity of elements for every tuple
        for idx, item in enumerate(data):
            rank = ""
            for lst in pfi:
                if lst[0].lower() in item:
                    rank = lst[1]
                    break
            data[idx] = item + (rank,)

In [43]:
#Human Development Index
#source - https://hdr.undp.org/data-center/human-development-index#/indicies/HDI

#skiping useless rows without data
skip_rows = 5  # Skip the first 5 rows

df = pd.read_excel('HDR23-24_Statistical_Annex_HDI_Table.xlsx', index_col=None, header=0, skiprows=skip_rows, usecols=["HDI rank", "Country", "Value"])
df_filtered = df[pd.to_numeric(df[df.columns[0]], errors='coerce').notnull()]
for idx, item in enumerate(data):
    rank = 0
    country = ""
    value = ""
    for index, row in df_filtered.iterrows():
        if row["Country"].lower() in item:
            rank = row["HDI rank"]
            value = row["Value"]
            break
    if rank == 0:
        data[idx] = item + (('N/A'),)
    else:
        data[idx] = item + ((str(value) + " (rank: " + str(int(rank)) + ")"),)

In [44]:
#Corruption Perceptions Index
#source - https://www.transparency.org/en/cpi/2023

#skiping useless rows without data
skip_rows = 3

df = pd.read_excel('CPI2023_Global_Results__Trends.xlsx',sheet_name="CPI 2023", index_col=None, header=0, skiprows=skip_rows, usecols=["Country / Territory", "CPI score 2023", "Rank"])
for idx, item in enumerate(data):
    country = ""
    cpi = ""
    rank = 0
    for index, row in df.iterrows():
        if row["Country / Territory"].lower() in item:
            cpi = row["CPI score 2023"]
            rank = row["Rank"]
            break
    if rank == 0:
        data[idx] = item + (('N/A'),)
    else:
        data[idx] = item + ((str(cpi) + " (rank: " + str(rank) + ")"),)

In [45]:
print(data)

[('afghanistan', '', '', <psycopg2.extensions.Binary object at 0x000001F748E06F10>, 'Kabul', '', 'Pashto, Dari', '', 'Afghani (افغانى) (AFN)', 'UTC+4:30Lunar Calendar (Afghanistan Time)', '', '42,239,854', '-65,846', '352.6', '-6.2%', '14.4%', '2.3%', '0.2', '85.3%', '', '18%', '160', '6 Not Free', '1', '5', '19.09 (world place 178)', '0.462 (rank: 182)', '20 (rank: 162)'), ('albania', '', '', <psycopg2.extensions.Binary object at 0x000001F74B8C33C0>, 'Tirana', '', 'Albanian', '', 'Lek (ALL)', 'UTC+1 (CET)', '+355', '2,745,972', '-8,000', '8,367.8', '3.4%', '11.6%', '6.7%', '1.5', '100.0%', '0.0%', '83%', '42', '68 Partly Free', '28', '40', '54.1 (world place 99)', '0.789 (rank: 74)', '37 (rank: 98)'), ('algeria', '', '', <psycopg2.extensions.Binary object at 0x000001F74C690E70>, 'Algiers', '', 'Arabic, Berber', '', 'Algerian dinar (DZD)', 'UTC+1 (CET)', '+213', '45,606,480', '-9,999', '5,260.2', '4.1%', '11.8%', '9.3%', '3.7', '100.0%', '0.1%', '71%', '90', '32 Not Free', '10', '22', 

In [46]:
data_list = list(data)
#deleting lists with empy data on 5th symbol (because in this case we got no base info from wiki) and lists with less than 27 elements
filtered_data = [lst for lst in data_list if len(lst) > 27]
filtered_data = [lst for lst in filtered_data if lst[4] != '']
modified_data = tuple(filtered_data)
print(modified_data)

(('afghanistan', '', '', <psycopg2.extensions.Binary object at 0x000001F748E06F10>, 'Kabul', '', 'Pashto, Dari', '', 'Afghani (افغانى) (AFN)', 'UTC+4:30Lunar Calendar (Afghanistan Time)', '', '42,239,854', '-65,846', '352.6', '-6.2%', '14.4%', '2.3%', '0.2', '85.3%', '', '18%', '160', '6 Not Free', '1', '5', '19.09 (world place 178)', '0.462 (rank: 182)', '20 (rank: 162)'), ('albania', '', '', <psycopg2.extensions.Binary object at 0x000001F74B8C33C0>, 'Tirana', '', 'Albanian', '', 'Lek (ALL)', 'UTC+1 (CET)', '+355', '2,745,972', '-8,000', '8,367.8', '3.4%', '11.6%', '6.7%', '1.5', '100.0%', '0.0%', '83%', '42', '68 Partly Free', '28', '40', '54.1 (world place 99)', '0.789 (rank: 74)', '37 (rank: 98)'), ('algeria', '', '', <psycopg2.extensions.Binary object at 0x000001F74C690E70>, 'Algiers', '', 'Arabic, Berber', '', 'Algerian dinar (DZD)', 'UTC+1 (CET)', '+213', '45,606,480', '-9,999', '5,260.2', '4.1%', '11.8%', '9.3%', '3.7', '100.0%', '0.1%', '71%', '90', '32 Not Free', '10', '22', 

In [47]:
for item in modified_data:
    if len(item) < 28:
        print(item)

In [48]:
for item in modified_data:
    if item[11] == '':
        print(item)

('state of palestine', '', '', <psycopg2.extensions.Binary object at 0x000001F749A20D50>, 'Jerusalem', 'Gaza City', 'Arabic', '', 'Egyptian pound (EGP), Israeli new shekel (ILS), Jordanian dinar (JOD), (de facto)', 'UTC+2 (Palestine Standard Time)', '+970', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', 'N/A', 'N/A')
('vatican city', '', '', <psycopg2.extensions.Binary object at 0x000001F74EC5BD20>, 'city-state', '', 'Italian', '', 'Euro (€) (EUR)', 'UTC+1 (CET)', '+379', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', 'N/A', 'N/A')


In [49]:
# replace simpols /n and \xa0/
replaced_tuples = []
for tuple_item in modified_data:
    replaced_strings = []
    for idx, string in enumerate(tuple_item):
        # Check if replacements are needed
        if idx!=3 and "\n" in string:
            string = string.replace("\n", "")
        if idx!=3 and "\xa0/" in string:
            string = string.replace("\xa0/", ",")
        replaced_strings.append(string)
    replaced_tuples.append(tuple(replaced_strings))

### Writing data into DB

In [51]:
insert_query = """
INSERT INTO country_data (
    country, country_second_name, country_third_name, img, capital, largest_city, official_languages,
    common_languages, currency, time_zone, calling_code, population, net_migration, gdp_per_capita, gdp_growth,
    unemployment, inflation, co2_emissions, access_to_electricity, electricity_production, access_to_internet,
    gpi, total_score_and_status, political_rights, civil_liberties, pfi, hdi, cpi
) VALUES %s
"""

template = """
(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

psycopg2.extras.execute_values (
    cur, insert_query, replaced_tuples, template=template, page_size=100
) 
base.commit()

### Climate info

In [3]:
# basic climate info
climate_info = []
headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) '
                      'Chrome/39.0.2171.95 Safari/537.36'}
async with ClientSession() as session:
    async with session.get("https://www.cia.gov/the-world-factbook/field/climate/",
                           headers=headers) as countries_wiki:
        
        html = BS(await countries_wiki.text(), 'lxml')
        info = html.find('main').find_all('div', class_='row')[1]
        for div in info.find_all('div'):
            if div.find('a') is not None and div.find('p') is None:
                
                country = div.find('a').text.title()
                main_info = div.text[len(country):]
                climate_info.append((country, main_info))

In [4]:
# average yearly temperature
ayt = ""
updated_climate_info = []
headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) '
                      'Chrome/39.0.2171.95 Safari/537.36'}
async with ClientSession() as session:
    async with session.get("https://en.wikipedia.org/wiki/List_of_countries_by_average_yearly_temperature",
                           headers=headers) as countries_wiki:
        
        html = BS(await countries_wiki.text(), 'lxml')
        info = html.find('table').find('tbody')
        tag_to_exclude = info.find_all('sup')
        for tag in tag_to_exclude:
            tag.decompose()
        for tr in info.find_all('tr'):
            if tr is not None and tr.find('td') is not None:
                country = tr.find_all('td')[1].text.replace('\xa0', '')
                ayt = tr.find_all('td')[2].text.replace('\xa0', '').replace('\n', '')
                for item in climate_info:
                    if country.title() == item[0]:
                        if ayt != '':
                            new_info = item[1] + "\n" + "Average yearly temperature " + ayt
                            updated_climate_info.append((item[0], new_info))

In [5]:
# average annual precipitation
aap = ""
updated_climate_info2 = []
headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) '
                      'Chrome/39.0.2171.95 Safari/537.36'}
async with ClientSession() as session:
    async with session.get("https://en.wikipedia.org/wiki/List_of_countries_by_average_annual_precipitation",
                           headers=headers) as countries_wiki:
        
        html = BS(await countries_wiki.text(), 'lxml')
        info = html.find('table').find('tbody')
        tag_to_exclude = info.find_all('sup')
        for tag in tag_to_exclude:
            tag.decompose()
        for tr in info.find_all('tr'):
            if tr is not None and tr.find('td') is not None:
                country = tr.find_all('td')[1].text.replace('\xa0', '')
                aap = tr.find_all('td')[2].text.replace('\xa0', '').replace('\n', '')
                for item in updated_climate_info:
                    if country.title() == item[0]:
                        if aap != '':
                            new_info = item[1] + "\n" + "Average annual precipitation " + aap + " mm/year"
                            updated_climate_info2.append((item[0], new_info))

In [13]:
for item in updated_climate_info2:
    query = """
        UPDATE country_data 
        SET climate = %s 
        WHERE country = %s OR country_second_name = %s OR country_third_name = %s
    """
    params = (item[1].replace('\n', '\\n'), item[0].lower(), item[0].lower(), item[0].lower())   
    cur.execute(query, params)
    base.commit()

### Main cities

In [14]:
# info about main cities
countries = []

query = ("SELECT country "
         "FROM public.country_data ")
cur.execute(query)
for i in cur.fetchall():
    countries.append(i[0])

headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) '
                      'Chrome/39.0.2171.95 Safari/537.36'}
data = []
for country in countries:
    country_cities = ""
    async with ClientSession() as session:
        async with session.get("https://www.worlddata.info/europe/" + country.replace(" ", "-") + "/index.php",
                               headers=headers) as country_wiki:
            html = BS(await country_wiki.text(), 'lxml')
            info = html.find('div', class_='boxwhite floater')
            for i in info.find_all('table', class_='std100 hover'):
                if i.find('tr').find("city") != -1:
                    for row in i.find_all('tr'):
                        if row.find('td') is not None:
                            country_cities = country_cities + "\\n" + row.find('td').text
    if country_cities != "":
        cur.execute(
            "UPDATE country_data SET main_cities = %s WHERE country = %s", (country_cities, country))
        base.commit()

### Expanses

In [44]:
data = {}
new_str = ""
headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) '
                  'Chrome/39.0.2171.95 Safari/537.36'}
async with ClientSession() as session:
    async with session.get("https://www.theglobaleconomy.com/rankings/cost_of_living_wb/",
                           headers=headers) as country_wiki:
        html = BS(await country_wiki.text(), 'lxml')
        info = html.find('table', id='benchmarkTable').find('tbody')
        for i in info.find_all('tr'):
            country = i.find_all('td')[0].text.lower().replace('\xa0', '').replace('\n', '')
            new_str = "\n Cost of living index " + str(i.find_all('td')[1].text).strip().replace('\xa0', '').replace('\n', '') + " Global rank " + str(
                i.find_all('td')[2].text).strip().replace('\xa0', '').replace('\n', '')
            data[country] = new_str

async with ClientSession() as session:
    async with session.get("https://www.theglobaleconomy.com/rankings/food_price_index_wb/",
                           headers=headers) as country_wiki:
        html = BS(await country_wiki.text(), 'lxml')
        info = html.find('table', id='benchmarkTable').find('tbody')
        for i in info.find_all('tr'):
            country = i.find_all('td')[0].text.lower().replace('\xa0', '').replace('\n', '')
            new_str = data.get(country) + "\n Food price index " + str(i.find_all('td')[1].text).strip().replace('\xa0', '').replace('\n', '') + " Global rank " + str(
                i.find_all('td')[2].text).strip().replace('\xa0', '').replace('\n', '')
            data[country] = new_str

async with ClientSession() as session:
    async with session.get("https://www.theglobaleconomy.com/rankings/clothing_and_footwear_prices_wb/",
                           headers=headers) as country_wiki:
        html = BS(await country_wiki.text(), 'lxml')
        info = html.find('table', id='benchmarkTable').find('tbody')
        for i in info.find_all('tr'):
            country = i.find_all('td')[0].text.lower().replace('\xa0', '').replace('\n', '')
            new_str = data.get(country) + "\n Clothing and footwear prices " + str(i.find_all('td')[1].text).strip().replace('\xa0', '').replace('\n', '') + \
                    " Global rank " + str(i.find_all('td')[2].text).strip().replace('\xa0', '').replace('\n', '')
            data[country] = new_str

async with ClientSession() as session:
    async with session.get("https://www.theglobaleconomy.com/rankings/housing_and_utilities_price_index_wb/",
                           headers=headers) as country_wiki:
        html = BS(await country_wiki.text(), 'lxml')
        info = html.find('table', id='benchmarkTable').find('tbody')
        for i in info.find_all('tr'):
            country = i.find_all('td')[0].text.lower().replace('\xa0', '').replace('\n', '')
            new_str = data.get(country) + "\n Housing and utilities price index " + str(
                i.find_all('td')[1].text).strip().replace('\xa0', '').replace('\n', '') + \
                    " Global rank " + str(i.find_all('td')[2].text).strip().replace('\xa0', '').replace('\n', '')
            data[country] = new_str

async with ClientSession() as session:
    async with session.get("https://www.theglobaleconomy.com/rankings/healthcare_prices_wb/",
                           headers=headers) as country_wiki:
        html = BS(await country_wiki.text(), 'lxml')
        info = html.find('table', id='benchmarkTable').find('tbody')
        for i in info.find_all('tr'):
            country = i.find_all('td')[0].text.lower().replace('\xa0', '').replace('\n', '')
            new_str = data.get(country) + "\n Healthcare price index " + str(i.find_all('td')[1].text).strip().replace('\xa0', '').replace('\n', '') + " Global rank " + \
                    str(i.find_all('td')[2].text).strip().replace('\xa0', '').replace('\n', '')
        data[country] = new_str

async with ClientSession() as session:
    async with session.get("https://www.theglobaleconomy.com/rankings/education_prices_wb/",
                           headers=headers) as country_wiki:
        html = BS(await country_wiki.text(), 'lxml')
        info = html.find('table', id='benchmarkTable').find('tbody')
        for i in info.find_all('tr'):
            country = i.find_all('td')[0].text.lower().replace('\xa0', '').replace('\n', '')
            new_str = data.get(country) + "\n Education service price index " + str(i.find_all('td')[1].text).strip().replace('\xa0', '').replace('\n', '') + \
                    " Global rank " + str(i.find_all('td')[2].text).strip().replace('\xa0', '').replace('\n', '')
        data[country] = new_str

async with ClientSession() as session:
    async with session.get("https://www.theglobaleconomy.com/rankings/transport_prices_wb/",
                           headers=headers) as country_wiki:
        html = BS(await country_wiki.text(), 'lxml')
        info = html.find('table', id='benchmarkTable').find('tbody')
        for i in info.find_all('tr'):
            country = i.find_all('td')[0].text.lower().replace('\xa0', '').replace('\n', '')
            new_str = data.get(country) + "\n Transport prices index " + str(i.find_all('td')[1].text).strip().replace('\xa0', '').replace('\n', '') + " Global rank " + \
                        str(i.find_all('td')[2].text).replace('\xa0', '').replace('\n', '').strip()
            data[country] = new_str


for country, expanses in data.items():
    cur.execute(
        "UPDATE country_data SET expanses = %s WHERE country = %s or country_second_name = %s or country_third_name = %s", (expanses, country, country, country))
    base.commit()

### Taxes

In [31]:
data = []
headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) '
                  'Chrome/39.0.2171.95 Safari/537.36'}
async with ClientSession() as session:
    async with session.get("https://en.wikipedia.org/wiki/List_of_countries_by_tax_rates",
                           headers=headers) as wiki:
        html = BS(await wiki.text(), 'lxml')
        info = html.find_all('table')[1]
        tag_to_exclude = info.find_all('sup')
        for tag in tag_to_exclude:
            tag.decompose()
        tab = info.find('tbody')
        for i in tab.find_all('tr'):
            if i.find('td') is not None and len(i.find_all('td')) > 4:
                country = i.find_all('td')[0].text.replace('\xa0', '').replace('\n', '')
                new_str = ""
                company_taxes = i.find_all('td')[1].text.replace('\xa0', '').replace('\n', '')
                if company_taxes != "":
                    new_str += "Company taxes: " + company_taxes + "\\n"
                lmr = i.find_all('td')[2].text.replace('\xa0', '').replace('\n', '')
                if lmr != "":
                    new_str = += "Lowest marginal rate: " + lmr + "\\n"
                hmr = i.find_all('td')[3].text.replace('\xa0', '').replace('\n', '')
                if hmr != "":
                    new_str += "Highest marginal rate: " + hmr + "\\n"
                vat = i.find_all('td')[4].text.replace('\xa0', '').replace('\n', '')
                if vat != "":
                    new_str += "Value-added tax or goods and services tax: " + vat + "\\n"
                data.append((country, new_str))

for item in data:
    cur.execute(
        "UPDATE country_data SET taxes = %s WHERE country = %s", (item[1], item[0].lower()))
    base.commit()